# Clustering Crypto

In [6]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Fetching Cryptocurrency Data

In [7]:

# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [8]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
r = requests.get(url)
data = r.json()

In [9]:

list(data.keys())

['Response',
 'Message',
 'Data',
 'BaseImageUrl',
 'BaseLinkUrl',
 'RateLimit',
 'HasWarning',
 'Type']

In [10]:
data = data['Data']

In [11]:
crypto_df = pd.DataFrame.from_dict(data)

In [12]:
crypto_df = crypto_df.T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [14]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
611,20909,/coins/611/overview,/media/35650940/611-sixeleven.png,1465914773,611,611,SixEleven,SixEleven (611),"611 is a Namecoin based cryptocurrency, and it...",N/A,...,0000-00-00,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ADA,321992,/coins/ada/overview,/media/37746235/ada.png,1507247613,ADA,ADA,Cardano,Cardano (ADA),Designed and developed by IOHK in conjunction ...,N/A,...,2017-10-05,45000000000,0,0,0,blockchain,NaN,NaN,NaN,NaN
LTC,3808,/coins/ltc/overview,/media/37746243/ltc.png,1426004323,LTC,LTC,Litecoin,Litecoin (LTC),Litecoin LTC - provides faster transaction con...,N/A,...,2011-10-13,84000000,0,0,0,blockchain,NaN,NaN,8,9.15186e+06
AXS,937569,/coins/axs/overview,/media/37746925/axs.png,1604484236,AXS,AXS,Axie Infinity Shards,Axie Infinity Shards (AXS),What are Axie Infinity Shards (AXS)?Axie Infin...,N/A,...,2020-10-27,-1,0,0,1,token,ETH,0xf5d669627376ebd411e34b98f19c868c8aba5ada,18,NaN
TRX,310829,/coins/trx/overview,/media/37746879/trx.png,1506465755,TRX,TRX,TRON,TRON (TRX),TRON is a cryptocurrency payment platform. It ...,N/A,...,2017-09-26,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN


In [15]:

# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [16]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [17]:
# Remove the 'IsTrading' column
crypto_df = crypto_df.drop(columns = 'IsTrading')

In [18]:
# Remove rows with at least 1 null value
crypto_df.isnull().sum()

Id                         0
Url                        0
ImageUrl                   0
ContentCreatedOn           0
Name                       0
Symbol                     0
CoinName                   0
FullName                   0
Description                0
AssetTokenStatus           0
Algorithm                  0
ProofType                  0
SortOrder                  0
Sponsored                  0
Taxonomy                   0
Rating                     0
TotalCoinsMined         1209
BlockNumber             1209
NetHashesPerSecond      1212
BlockReward             1209
BlockTime               1209
AssetLaunchDate         1209
MaxSupply               1209
MktCapPenalty           1209
IsUsedInDefi            1209
IsUsedInNft             1209
PlatformType            1360
BuiltOn                 1469
SmartContractAddress    1472
DecimalPoints           1461
Difficulty              1455
dtype: int64

In [19]:
crypto_df = crypto_df.dropna()
crypto_df.isnull().sum()

Id                      0
Url                     0
ImageUrl                0
ContentCreatedOn        0
Name                    0
Symbol                  0
CoinName                0
FullName                0
Description             0
AssetTokenStatus        0
Algorithm               0
ProofType               0
SortOrder               0
Sponsored               0
Taxonomy                0
Rating                  0
TotalCoinsMined         0
BlockNumber             0
NetHashesPerSecond      0
BlockReward             0
BlockTime               0
AssetLaunchDate         0
MaxSupply               0
MktCapPenalty           0
IsUsedInDefi            0
IsUsedInNft             0
PlatformType            0
BuiltOn                 0
SmartContractAddress    0
DecimalPoints           0
Difficulty              0
dtype: int64

In [20]:

# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [21]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']

In [22]:

# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coinname_df = pd.DataFrame(
    data = crypto_df, columns = ['CoinName'])

In [23]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns = 'CoinName')
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,FullName,Description,AssetTokenStatus,Algorithm,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty


In [24]:

# Create dummy variables for text features
crypto_dummies= pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_dummies.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,FullName,Description,AssetTokenStatus,SortOrder,...,AssetLaunchDate,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty


In [ ]:
# Standardize data



### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components



In [ ]:
# Create a DataFrame with the principal components data



### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [25]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_dummies)
print(crypto_scaled[0:1])

ValueError: Found array with 0 sample(s) (shape=(0, 28)) while a minimum of 1 is required by StandardScaler.

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features



### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters



#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos



In [ ]:
# Print the total number of tradable cryptocurrencies

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot



In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

